В коде реализован симметричный алгоритм Гаусса. Цель — быстрая проверка правильности шагов диагонализации матрицы билинейной формы и поиск матрицы перехода от исходного базиса к базису, в котором матрица имеет диагональный вид. Алгоритм преполагает, что пользователь:
- использует операции только с целыми числами
- знает, что умножение на ноль делает любой алгоритм бессмысленным.

В силу:
- $\beta$ — билинейная форма, $B$ — матрица билинейной формы
- $\mathbb{e}' = \mathbb{e}C$
- $B' = C^TBC$

Тогда $(B|E) \rightarrow (D|C)$, где C — матрица преобразований, к которой применены только элементарные преобразования (ЭП) столбцов, а D — диагональная матрица, к которой применялись ЭП и строк, и столбцов. Получаем, что C — искомая матрица перехода в новому базису.

ЭП — элем. преобразование строк и столбцов (всего 3 типа).

Список команд ЭП:
- ```<1> <i> <j> <coeff>``` — к i-й строке прибавляется j-я строка, умноженная на coeff (то же происходит одновременно и со столбцов, так как это симметричная операция)
- `<2> <i> <j>` — меняются местами i-я и j-я строки
-`<3> <i> <coeff>` — i-я строка умножается на ненулевой coeff

Другие команды:
- `<0>` — остановить ввод команд
- `<4> <1>` — отменить последнюю команду, возвращает предыдущий вид матрицы
- `<4> <2>` — показать все шаги по преобразованию матриц

In [95]:
import numpy as np
from copy import deepcopy

In [118]:
def showMM(to_show:list, do = True):
    a = ['step', 'A', 'E']
    if do:
        for i in range(3):
            print(f"{a[i]} = {to_show[i]}")


In [119]:
def restoreMatrix(all_steps:list, *arg, pr = True):
    """параметр pr отвечает за вывод матриц"""
    if arg[0] == 'undo':
        # вернуть и вывести предпосл шаг
        all_steps.pop()
        print('undo last step')
        showMM(all_steps[-1], pr)
        return all_steps[-1]
    elif arg[0] == 'steps':
        # получить все шаги
        if pr == True:
            c = 0
            for step in all_steps:
                c+=1
                print(f"{c}:{step[0]}", end=' ')
            print()
        return [step[0] for step in all_steps]

# all_steps = [([2, 1, 2], np.array([[2, 1, 3],[1, 0, 1],[3, 1, 4]]), np.array([[0, 1, 0], [1, 0, 0],[0, 0, 1]])),
#              ([3, 2, 2], np.array([[2, 2, 3],[2, 0, 2],[3, 2, 4]]), np.array([[0, 2, 0], [1, 0, 0],[0, 0, 1]]))]



In [138]:
"""Сюда введите нужную Вам матрицу A:"""
A = np.array([[0, 1, 1],
              [1, 2, 3],
              [1, 3, 4]], dtype = float)

all_steps=[]
def algo(A, pr=True):
    """
    матрица A — к ней будут применяться преобразования,
    матрица E — матрица преобразований,
    pr — параметр, который задает вывод промежуточных шагов
    """
    E = np.eye(A.shape[0])
    while True:
        try:
            step = list(map(int, input().split()))
        except ValueError as e:
            raise e("Некорректный ввод значений. Попробуйте ввести значения через пробел")
        except TypeError as e:
            raise e("Некорректный ввод типа. Используйте целые числа")
        if step[0] == 0:
            print('results: A, E')
            print(A)
            print(E)
            break
        if step[0] == 1:
            str1 = step[1]-1
            str2 = step[2]-1
            k = step[3]
            A[str1] = A[str1]+A[str2]*k
            A[:, str1] = A[:, str1] + A[:, str2]*k
            E[:, str1] = E[:, str1] + E[:, str2]*k
        elif step[0] == 2:
            str1 = step[1]-1
            str2 = step[2]-1
            temp_str = A[str1].copy()
            A[str1]=A[str2]
            A[str2] = temp_str
            temp_str = A[:, str1].copy()
            A[:, str1]=A[:, str2]
            A[:, str2]= temp_str
            temp_str = E[:, str1].copy()
            E[:, str1] = E[:, str2]
            E[:, str2] = temp_str
        elif step[0] == 3:
            str1 = step[1]-1
            k = step[2]
            A[str1] = A[str1]*k
            A[:, str1]=A[:, str1]*k
            E[:, str1] = E[:, str1]*k
        if step[0] != 0 and step[0]!=4:
            all_steps.append((step, np.array(A, copy=True), np.array(E, copy=True)))
            if pr == True:
                # if step[0]==1:
                #     print(f"Э_{step[0]}({step[1]}, {step[2]}, {step[3]})")
                # else:
                #     print(f"Э_{step[0]}({step[1]}, {step[2]})")
                print(A)
                print(E)
        if step[0] == 4:
            # to restore matrix (undo) or see last steps
            my_dict = {1: 'undo', 2: 'steps'}
            restored = restoreMatrix(all_steps, my_dict[step[1]])
            if step[1] == 1:
                A, E = restored[1].copy(), restored[2].copy()


algo(A)



0
results: A, E
[[0. 1. 1.]
 [1. 2. 3.]
 [1. 3. 4.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
